# Modeling the scara robot
Using the Denavit-hartenberg convention and the Kinematics and Dynamics Library. For documentation on kdl see

https://www.orocos.org/kdl/user-manual

and

http://docs.ros.org/en/indigo/api/orocos_kdl/html/

<img src="https://global.yamaha-motor.com/business/robot/lineup/ykxg/large/img/index/img_l.jpg" width=500 />
(Source: yamaha-motor.com)

### Preliminary: Install kdl on google colab
Skip the next cell if you are running this notebook on a system that already has kdl installed.

In [ ]:
!git clone https://github.com/orocos/orocos_kinematics_dynamics.git
!apt install libeigen3-dev libcppunit-dev # Without sudo on colab
!mkdir orocos_kinematics_dynamics/orocos_kdl/build
%cd orocos_kinematics_dynamics/orocos_kdl/build
!cmake ..
!make
!make install
%cd ../../python_orocos_kdl
!git submodule update --init
!mkdir build
%cd build
!cmake ..
!make
!make install
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib
!ldconfig
!python3 ../tests/PyKDLtest.py

In [ ]:
import numpy as np
import PyKDL as kdl
import doctest

In [ ]:
class Scara:
    '''
    Defines a scara robot of 4dof.
    '''
    def __init__(self, l0=1, l1=1, l2=1):
        '''
        Arguments
        ---------
        l0 : float
           Height of the base
        l1, l2 : float
           Lengths of link 1 and 2, respectively

        Returns
        -------
        sc : kdl.Chain
           Representation of the robot as a chain of segments 
        '''
        # D-H table
        # Name  type   a   alpha  d  theta 
        dh_table = [ ('link1', kdl.Joint.RotZ, l1, 0, l0, 0),
                    ('link2', , , , , ),
                    ('link3', , , , , ),
                    ('tool', , , , , )]

        self.chain = kdl.Chain()

        for name, jtype, a, alpha, d, theta in dh_table:
            self.chain.addSegment(kdl.Segment(name=name,
                                     joint=kdl.Joint(type=jtype),
                                     f_tip=kdl.Frame.DH(a, alpha, d, theta)))
        
        self.fksolver = kdl.ChainFkSolverPos_recursive(self.chain)
        
    def fk(self, jc):
        '''
        Arguments
        ---------
        jc : array-like
           The joint coordinates (angles)
           
        Returns
        -------
        Te : The frame of the end effector
        
        Tests
        -----
        Define robot. Check forward kinematics
        >>> sc = Scara()
        >>> # 1) In reference configuration
        >>> tcp0w = kdl.Vector(2,0,1) # In the world frame
        >>> tcp0 = kdl.Vector(0, 0, 0) # In the end-effector frame
        >>> tcp = sc.fk([0,0,0,0])*tcp0
        >>> print("TCP: x=%0.2f, y=%0.2f, z=%0.2f" %(tcp.x(), tcp.y(), tcp.z()))
        TCP: x=2.00, y=0.00, z=1.00
        >>> # 2) Rotate first two joints 90 degrees
        >>> tcp = sc.fk([np.pi/2, np.pi/2, 0, 0])*tcp0
        >>> print("TCP: x=%0.2f, y=%0.2f, z=%0.2f" %(tcp.x(), tcp.y(), tcp.z()))
        TCP: x=-1.00, y=1.00, z=1.00
        >>> # 3) Rotate third joint -90 degrees extend last joint 0.5
        >>> Te = sc.fk([0, 0, np.pi/2, 0.5])
        >>> tcp = Te*tcp0
        >>> print("TCP: x=%0.2f, y=%0.2f, z=%0.2f" %(tcp.x(), tcp.y(), tcp.z()))
        TCP: x=2.00, y=-0.00, z=0.50
        >>> x_e = kdl.Vector(0.5, 0, 0)
        >>> x_s = Te*x_e
        >>> print("x=%0.2f, y=%0.2f, z=%0.2f" %(x_s.x(), x_s.y(), x_s.z()))
        x=2.00, y=0.50, z=0.50
        
        '''
        
        q = kdl.JntArray(4)
        for i in range(4):
            q[i] = jc[i]
        ee_frame = kdl.Frame()
        self.fksolver.JntToCart(q, ee_frame)
        return ee_frame
     

In [ ]:
doctest.run_docstring_examples(Scara.fk, globals(), verbose=True)